In [231]:
import pandas as pd
pd.set_option('max_colwidth', 100)
import numpy as np
from collections import Counter

### Read data

In [172]:
raw = pd.read_csv('./Batch_4218958_batch_results.csv')

### Flatten data

In [173]:
data = []
for i,row in raw.iterrows():
    for j in range(1, 13):
        query = row['Input.q%d' % j]
        response = row['Input.r%d' % j]
        gender = row['Input.g%d' % j]
        
        if row['Answer.answer%d' % j] is not np.nan:
            answer = row['Answer.answer%d' % j]
        else:
            answer = row['Answer.answer%d.label' % j]
        
        data.append((row['HITId'], j, row['WorkerId'], gender, query, response, answer))

In [174]:
melted = pd.DataFrame(data)
melted.columns = ['HITId', 'q_idx', 'WorkerId', 'gender', 'query', 'response', 'answer']

In [175]:
# include spammers:
include = False
if not include:
    spammers = ['A1NE8EMAHNHLQ6']
    melted = melted[~melted.WorkerId.isin(spammers)].copy()

In [176]:
data = []
for i, group in melted.groupby(['HITId', 'q_idx']):
    row = []
    row.append(group.iloc[0]['HITId'])
    row.append(group.iloc[0]['gender'])
    row.append(group.iloc[0]['query'])
    row.append(group.iloc[0]['response'])
    for j in range(0,5):
        if j < len(group):
            row.append(group.iloc[j]['WorkerId'])
            row.append(group.iloc[j]['answer'])
    data.append(row)

In [177]:
flat = pd.DataFrame(data)
flat.columns = ['HITId', 'gender', 'query', 'response'] + \
    sum([ ['WorkerId%d' % i, 'answer%d' % i] for i in range(0,5) ], [])

In [178]:
flat.head(1)

,HITId,gender,query,response,WorkerId0,answer0,WorkerId1,answer1,WorkerId2,answer2,WorkerId3,answer3,WorkerId4,answer4
0,301KG0KX9CMNBG7QOX77G1G3QY52H5,M,Do I think Michigan wins ultimately?,"It's a toss up, but I have a feeling that Mich...",A7XVPJCI4Z8MC,Definitely yes,A1ET2J1PIP0RGO,Probably yes,AG1OG2FU9F09G,Probably yes,AB2JXV5YUJ1Y7,Probably yes,None,None


### Collapse labels

In [179]:
flat.answer0.unique()

array(['Definitely yes', 'Probably yes', 'Probably no',
       "No information or doesn't make sense", 'Definitely no'],
      dtype=object)

In [180]:
def i_no_i(x):
    if 'No information' in x:
        return 'ni'
    else:
        return 'i'
    
def reg(x):
    if 'Definitely yes' in x or 'Definitely no' in x:
        return 'f'
    elif 'Probably yes' in x or 'Probably no' in x:
        return 'p'
    else:
        return 'n'

### Interannotator agreement

In [181]:
order = ['f', 'p', 'n']
order = ['i', 'ni']

agree = 0
for i, row in flat.iterrows():
    labels = [ row['answer%d' %i ] for i in range(0, 5) if row['answer%d' %i ] != None ]
    labels = list(map(i_no_i, labels))
    unique, counts = np.unique(labels,return_counts=True)
    unique = unique.tolist()

    if any(counts >= 3):
        agree += 1

print(agree, len(flat))
print(agree / len(flat))

113 120
0.9416666666666667


In [185]:
order = ['i', 'ni']
#order = ['f', 'p', 'n']

j = 0
for i, row in flat.iterrows():
    labels = [ row['answer%d' %i ] for i in range(0, 5) if row['answer%d' %i ] != None ]
    labels = list(map(i_no_i, labels))
    unique, counts = np.unique(labels,return_counts=True)
    unique = unique.tolist()
    
    l = []
    for i in order:
        if i in unique:   
            idx = unique.index(i)
            count = counts[idx]
        else:
            count = 0
        l.append(str(count))
        
    if len(labels) == 4:
        j += 1
        print('\t'.join(l))
print(j)

4	0
4	0
4	0
4	0
4	0
1	3
4	0
4	0
4	0
4	0
4	0
4	0
4	0
4	0
4	0
4	0
1	3
3	1
4	0
4	0
3	1
4	0
4	0
4	0
4	0
4	0
4	0
3	1
4	0
4	0
2	2
4	0
3	1
4	0
4	0
4	0
4	0
4	0
3	1
4	0
4	0
4	0
2	2
2	2
1	3
4	0
4	0
4	0
4	0
4	0
4	0
4	0
3	1
2	2
4	0
4	0
4	0
4	0
4	0
3	1
4	0
4	0
2	2
4	0
4	0
2	2
3	1
4	0
4	0
2	2
3	1
4	0
72


non-collapsed:
42.58%
Kappa: 0.17

collapsed:
81.67%
Kappa: 0.63

### Majority labels

In [224]:
def majority(x, collapse=i_no_i, n=5):
    labels = [ x['answer%d' % i] for i in range(0, n)]
    labels = [ i for i in labels if i ]
    labels = list(map(collapse, labels))
    
    c = Counter(labels)
    commons = c.most_common(2)
    
    if len(commons) > 1 and commons[0][1] == commons[1][1]:
        return 'tie'
    else:
        return commons[0][0]

In [225]:
flat['binary_majority'] = flat.apply(majority, axis=1)
flat['majority'] = flat.apply(lambda x: majority(x, collapse=lambda x:x), axis=1)

In [226]:
for i, group in flat.groupby('gender'):
    print(i)
    print(group.majority.value_counts(normalize=False))

M
tie                                     18
Probably no                             10
Probably yes                            10
Definitely yes                           9
Definitely no                            7
No information or doesn't make sense     6
Name: majority, dtype: int64
W
Definitely no                           16
Definitely yes                          14
No information or doesn't make sense     8
Probably no                              8
tie                                      8
Probably yes                             6
Name: majority, dtype: int64


In [227]:
for i, group in flat.groupby('gender'):
    print(i)
    print(group.binary_majority.value_counts(normalize=False))

M
i      53
ni      4
tie     3
Name: binary_majority, dtype: int64
W
i      54
tie     4
ni      2
Name: binary_majority, dtype: int64


### Identifying spammers

In [46]:
for i, group in melted.groupby('WorkerId'):
    print('WorkerId: %s' % i)
    print(group.answer.value_counts())
    print()

WorkerId: A122GX6WP52RVZ
Definitely yes    18
Probably yes      13
Definitely no     12
Probably no        5
Name: answer, dtype: int64

WorkerId: A1ET2J1PIP0RGO
Definitely no                           21
Definitely yes                          20
Probably yes                            14
Probably no                             11
No information or doesn't make sense     6
Name: answer, dtype: int64

WorkerId: A1GS7JCUCVTLO2
Probably no       6
Probably yes      4
Definitely yes    1
Definitely no     1
Name: answer, dtype: int64

WorkerId: A1NE8EMAHNHLQ6
Probably yes      71
Definitely yes     1
Name: answer, dtype: int64

WorkerId: A1S6Y0P5M0HNWG
No information or doesn't make sense    7
Probably yes                            3
Probably no                             1
Definitely yes                          1
Name: answer, dtype: int64

WorkerId: A22VGT2F28LTWC
No information or doesn't make sense    4
Probably no                             4
Definitely yes                       

In [47]:
spammers = ['A1NE8EMAHNHLQ6']

### Qualitative analysis

In [232]:
flat[['gender', 'query', 'response']][flat.binary_majority == 'ni']

,gender,query,response
5,M,Didnt you hear what Norbert Haug & co. say?,What did he say?
12,W,Do you really think that every single woman is weaker than every single man?,I think you're misunderstanding.
40,M,Did I mention how Rank 22 for the Artifact Weapon costs 64.000 points?,I'm not sure what you're saying...
64,M,Did you even read the whole article?,Did you even read the article?
80,W,Did you know fast food drive thrus are on a timer?,Did you know that the sun rises in the east?
86,M,Do you know a guide or article on how emulators such as RPCS3 work?,I think he was trying to make a joke.


In [233]:
flat[['gender', 'query', 'response']][flat.binary_majority == 'tie']

,gender,query,response
54,M,Do you *really* think one needs to sacrifice the (admittedly-small-but-not-zero) performance off...,I'm not sure. I don't know how the process works and that's a very large amount of work.
78,W,Did you even read the website?,"It's a joke, I know you are joking, I'm just saying it's a bad idea."
79,M,Do you need to ally yourself with a group that could cause later blowback?,It's a hypothetical.
101,M,Do you think your company will/can consistently improve their stock price?,I don't have a good understanding of what stock price is and what it should be. I'm not a stock ...
110,W,Did you want to put Top Gear on while we review our retirement portfolios?,I think I will go for a walk and just chill in the car.
113,W,Did you want to put Top Gear on while we review our retirement portfolios?,"I'm just gonna take this as a compliment, and go about my day."
117,W,Did you not have some basic research skills course in high school at least?,I was a pre med student.


In [234]:
flat[['gender', 'query', 'response']][flat.binary_majority == 'i'].sample(10)

,gender,query,response
18,M,Did you really think that NATO/EU expansionism up to Russian borders would occur without some ci...,"The Ukrainians have been fighting with Russia for years, it's only a matter of time."
49,M,Are you that dense that you think referendums should be held yearly until you get the result you...,"I was being sarcastic, but yeah."
71,W,Doesnt that kind of mean the dogs baseline personality isnt wonderful?,I'm pretty sure that is what it means
37,M,Do you think that a football team uses more people through the course of a season than a basebal...,"No, but the average person spends a lot of money on a sports team."
65,M,Are you getting yellow on the 3DS?,"Yes. It's a bit annoying, but the blue screen is annoying and it's the only way I can use the ga..."
33,W,Do people really feel its a racial law?,"No, but there's nothing wrong with using it. I personally have no problem with it either."
38,M,Do you know what would happen if we went to war with Russia?,I do not. I've not been involved.
114,W,Do you know why no amount of disputable studies on IQ will ever affect my own self worth as a wo...,"Because it's not a measure of intelligence, it's a measure of emotional maturity."
34,M,Are the Knicks going to be a good team next year?,I think they'll be a good team this season.
56,M,Did you really think that NATO/EU expansionism up to Russian borders would occur without some ci...,"You are not wrong, but you can never count on a single person to be wrong."
